In [1]:
# What did I learned so far:
# 1) you do the best for the mean encoding of items in stock and shops open
#  and train a separate model for those
# 2) for the rest you force zero the discontinued items and closed shops
# 3) for the sparse shops and items you predict the mean of last tree month 

# Prepare data

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Load data
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

categories = pd.read_csv('../item_categories.csv')
items = pd.read_csv('../items.csv')

In [3]:
%store -r __prepare_data

In [4]:
%%time
__prepare_data

CPU times: user 1min 42s, sys: 44.8 s, total: 2min 27s
Wall time: 2min 42s


In [5]:
# aggregate daily sales to monthly sales
monthly = train_df.groupby(["item_id","shop_id","date_block_num"]).item_cnt_day.sum()
monthly = monthly.reset_index()
monthly.columns = ["item_id","shop_id","date_block_num","target"]

# this is still zero-suppressed data frame
monthly.head()

,item_id,shop_id,date_block_num,target
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0


# Explore sales
* find how many items didn't sell last 1, 2, and 3 months (discontinued items)
* find how many stores didn't sell anything last 1, 2, and 3 month (closed stores)
* find how many items were selling throught at lease one year
* find how many stores were open through at least one year
* check intersections

## Explore shop sales

prepare data for shops and for items in one go below

In [6]:
# unique shops, items, and dates
shops = pd.DataFrame({'shop_id': monthly.shop_id.unique()}).sort_values(by=['shop_id'])
items = pd.DataFrame({'item_id': monthly.item_id.unique()}).sort_values(by=['item_id'])
dates = pd.DataFrame({'date_block_num': range(35)}) # month #34 will be our prediction target
# trick to get an outer join in pandas
shops['dup_key'] = 1
items['dup_key'] = 1
dates['dup_key'] = 1

# create combinations of all shops and dates as well as combinations of all items and dates
scaffold_s = pd.merge(shops, dates, how='outer', on=['dup_key'])[['shop_id','date_block_num']]
scaffold_i = pd.merge(items, dates, how='outer', on=['dup_key'])[['item_id','date_block_num']]

# trade zero suppression for NaNs at dates with no sales, aggregate sales per shop of item
gapless_shops = pd.merge(scaffold_s,
                         (monthly                         
                          .groupby(['shop_id','date_block_num'])
                          .agg({'target': [sum,len,np.mean]}) # important to calculate before scaffold
                          .reset_index()),
                         how='left',
                         on=['shop_id','date_block_num'])

gapless_items = pd.merge(scaffold_i,
                         (monthly
                          .groupby(['item_id','date_block_num'])
                          .agg({'target': [sum,len,np.mean]}) # also makes sense to average over open shops only
                          .reset_index()),
                         how='left',
                         on=['item_id','date_block_num'])

gapless_shops.columns = ['shop_id', 'date_block_num', 'target_sum', 'target_len', 'target_mean']
gapless_items.columns = ['item_id', 'date_block_num', 'target_sum', 'target_len', 'target_mean']

# for presentation purposes pivot the "melted" table into [date,shop] sales matrix
sales_shops = gapless_shops.pivot(columns='shop_id', values='target_sum', index='date_block_num')

/usr/local/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:3947: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


Look into the shops

In [7]:
sales_shops.iloc[:,range(0,20)]

shop_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
date_block_num,,,,,,,,,,,,,,,,,,,,
0,5578.0,2947.0,1146.0,767.0,2114.0,NaN,3686.0,2495.0,1463.0,NaN,935.0,NaN,842.0,1178.0,1777.0,2724.0,2978.0,NaN,2856.0,3038.0
1,6127.0,3364.0,488.0,798.0,2025.0,877.0,4007.0,2513.0,1156.0,NaN,1026.0,NaN,1209.0,1036.0,1795.0,2746.0,2614.0,NaN,3341.0,2834.0
2,NaN,NaN,753.0,959.0,2060.0,1355.0,4519.0,2460.0,977.0,NaN,1017.0,NaN,1419.0,1245.0,1893.0,2835.0,3088.0,NaN,3551.0,3020.0
3,NaN,NaN,583.0,647.0,285.0,1008.0,3168.0,1540.0,-1.0,NaN,756.0,NaN,1364.0,1018.0,1247.0,2168.0,2118.0,NaN,2421.0,2291.0
4,NaN,NaN,553.0,710.0,1211.0,1110.0,3022.0,1647.0,NaN,NaN,683.0,NaN,917.0,1104.0,1489.0,1931.0,2261.0,NaN,2944.0,2089.0
5,NaN,NaN,832.0,784.0,1464.0,1393.0,3847.0,2085.0,NaN,NaN,764.0,NaN,1710.0,1358.0,1863.0,2419.0,3075.0,906.0,3144.0,2638.0
6,NaN,NaN,807.0,721.0,1378.0,1265.0,3360.0,2031.0,NaN,NaN,800.0,NaN,723.0,1094.0,1709.0,2599.0,2898.0,1044.0,2676.0,2459.0
7,NaN,NaN,875.0,890.0,1713.0,1510.0,3702.0,2397.0,NaN,NaN,755.0,NaN,1599.0,944.0,2216.0,2536.0,2457.0,1306.0,1778.0,2675.0
8,NaN,NaN,945.0,1026.0,1536.0,1298.0,4208.0,2599.0,NaN,NaN,1027.0,NaN,2032.0,1153.0,796.0,2790.0,1924.0,1459.0,2774.0,2801.0


Interesting:
* shops #0, #1, #8 (check for more) are closed for good -> I need to zero-out their forecast
* shop #9 opens exactly once a year and it'll be closed on month #34 (may also apply to items)

Let's look at more shops in case there are other interesting patterns

In [8]:
sales_shops.iloc[:,range(20,40)]

shop_id,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
date_block_num,,,,,,,,,,,,,,,,,,,,
0,NaN,1495.0,1924.0,1948.0,1768.0,8215.0,2331.0,5548.0,7057.0,2820.0,2978.0,10173.0,1238.0,NaN,NaN,2127.0,NaN,2399.0,1704.0,NaN
1,NaN,1854.0,2447.0,1941.0,1782.0,7934.0,2597.0,4546.0,5438.0,2834.0,2717.0,11160.0,1494.0,NaN,NaN,2205.0,NaN,2216.0,1670.0,NaN
2,NaN,2233.0,2588.0,2118.0,2014.0,9732.0,3036.0,5454.0,6918.0,3151.0,3026.0,12670.0,1890.0,NaN,NaN,2568.0,NaN,2629.0,2009.0,NaN
3,NaN,1846.0,2297.0,1698.0,787.0,7867.0,2381.0,3946.0,4910.0,2252.0,2379.0,9621.0,1364.0,NaN,NaN,1845.0,NaN,1946.0,1375.0,NaN
4,NaN,1646.0,1755.0,NaN,1967.0,6860.0,2148.0,3518.0,5703.0,2345.0,2304.0,9423.0,1209.0,NaN,NaN,2048.0,NaN,2019.0,1337.0,NaN
5,NaN,1912.0,2243.0,NaN,2599.0,8478.0,2661.0,275.0,7056.0,2803.0,2794.0,10072.0,1493.0,NaN,NaN,2280.0,NaN,1829.0,1539.0,NaN
6,NaN,1860.0,1903.0,NaN,2377.0,7361.0,2163.0,4148.0,6739.0,2205.0,2438.0,9500.0,93.0,NaN,NaN,2041.0,NaN,1293.0,1453.0,NaN
7,NaN,2061.0,2017.0,NaN,2304.0,7784.0,2452.0,4925.0,7521.0,2073.0,2799.0,9882.0,NaN,NaN,NaN,2391.0,NaN,1472.0,1895.0,NaN
8,NaN,2486.0,2518.0,NaN,3072.0,9251.0,3206.0,4981.0,6753.0,2832.0,3023.0,11204.0,NaN,NaN,NaN,2473.0,NaN,1666.0,1666.0,NaN


In [9]:
sales_shops.iloc[:,range(40,60)]

shop_id,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
date_block_num,,,,,,,,,,,,,,,,,,,,
0,NaN,1998.0,3991.0,1759.0,1708.0,1634.0,3363.0,2115.0,NaN,NaN,3406.0,2748.0,1812.0,2055.0,9386.0,NaN,3238.0,NaN,NaN,2017.0
1,NaN,1856.0,3965.0,2033.0,1823.0,1530.0,2660.0,1824.0,NaN,NaN,3054.0,2284.0,1737.0,1906.0,8075.0,NaN,2655.0,NaN,NaN,1897.0
2,NaN,1816.0,4953.0,2504.0,2131.0,1883.0,3103.0,1977.0,NaN,NaN,3610.0,2605.0,2034.0,2365.0,9488.0,NaN,3099.0,6586.0,3773.0,2028.0
3,NaN,1387.0,3978.0,1880.0,1628.0,1253.0,2160.0,1540.0,NaN,NaN,2740.0,1945.0,1446.0,1515.0,6726.0,NaN,2153.0,5493.0,2831.0,1388.0
4,NaN,1597.0,3594.0,1624.0,1379.0,1333.0,2522.0,1724.0,NaN,NaN,2839.0,2243.0,1482.0,1767.0,7006.0,852.0,2140.0,5233.0,2308.0,1374.0
5,NaN,2185.0,4349.0,2060.0,1666.0,1785.0,2865.0,1654.0,NaN,NaN,3761.0,2200.0,1815.0,2125.0,8101.0,728.0,3105.0,5987.0,2972.0,1707.0
6,NaN,1997.0,3801.0,1632.0,1487.0,1507.0,2966.0,1329.0,NaN,NaN,3235.0,2273.0,1616.0,1975.0,7225.0,795.0,3294.0,5827.0,3122.0,1747.0
7,NaN,2149.0,4035.0,1849.0,1610.0,1649.0,3315.0,1610.0,NaN,NaN,3718.0,2142.0,1772.0,2367.0,7792.0,719.0,3311.0,5413.0,2863.0,2048.0
8,NaN,2146.0,5093.0,2768.0,1849.0,1725.0,2963.0,1742.0,NaN,NaN,3655.0,1903.0,2125.0,2440.0,7827.0,812.0,2829.0,5213.0,2735.0,2008.0


Indeed, shops #0, #1, #8, #11, #13, #17, #23, #29, #30, #32, #33, #40, #43, #54
are closed for good can can be zeroed-out in the final forecast. But those could still be good in training for the periods when they open.

Shops #9, #20, #36 seems to open once a year in October, so those can also be zeroed-up for November.

There might be items with similar patterns, e.g. christmass or season related.

In [10]:
# find out if for previous 3, 2, and 1 consequetive months there were no sales

#  first, construct operators (matricies) that, applied to a time series, yield their lags:
#   1) create and flatten a unit matrix
current = np.identity(35).reshape(-1)
#   2) shift left by one, two, and tree positions, zero-pad on the right and zero-out top rows
lag1 = np.concatenate(([0]*35,  current[36:], [0])).reshape(-1,35)
lag2 = np.concatenate(([0]*70,  current[72:], [0]*2)).reshape(-1,35)
lag3 = np.concatenate(([0]*105, current[108:],[0]*3)).reshape(-1,35)

# replace NAs for 0 sales in the [date,shop] sales matrix
sales_shops.fillna(0, inplace=True)

sales_shops_lag1 = pd.DataFrame(np.dot(lag1, sales_shops.values), columns=shops.shop_id.tolist())
sales_shops_lag2 = pd.DataFrame(np.dot(lag2, sales_shops.values), columns=shops.shop_id.tolist())
sales_shops_lag3 = pd.DataFrame(np.dot(lag3, sales_shops.values), columns=shops.shop_id.tolist())

sales_shops_lag3.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5578.0,2947.0,1146.0,767.0,2114.0,0.0,3686.0,2495.0,1463.0,0.0,...,3406.0,2748.0,1812.0,2055.0,9386.0,0.0,3238.0,0.0,0.0,2017.0
4,6127.0,3364.0,488.0,798.0,2025.0,877.0,4007.0,2513.0,1156.0,0.0,...,3054.0,2284.0,1737.0,1906.0,8075.0,0.0,2655.0,0.0,0.0,1897.0


In [11]:
# Create a new surrogate boolean observable indicating if previous 3 months there were no sales
no_sales_shops = (sales_shops_lag1 + sales_shops_lag2 + sales_shops_lag3 < 1).stack().reset_index()
no_sales_shops.columns = ['date_block_num','shop_id','no_sales']
no_sales_shops = no_sales_shops.sort_values(by=['shop_id','date_block_num'])
no_sales_shops.head()

,date_block_num,shop_id,no_sales
0,0,0,True
60,1,0,False
120,2,0,False
180,3,0,False
240,4,0,False


In [12]:
# shops closed with no sales in the last 3 months
closed_shops = no_sales_shops[(no_sales_shops.date_block_num==34)&(no_sales_shops.no_sales==True)].shop_id.tolist()

print(closed_shops)

print("%d out of %d shops were not selling at least last 3 months" % \
      (sum(no_sales_shops[no_sales_shops.date_block_num==34].no_sales),len(shops)))

# one shop with negative sale (return) is also considered to be closed over last 3 months
monthly[monthly.shop_id==27].groupby(['date_block_num']).agg({'target': sum}).tail()

[0, 1, 8, 11, 13, 17, 23, 27, 29, 30, 32, 33, 40, 43, 54]
15 out of 60 shops were not selling at least last 3 months


,target
date_block_num,
27,3518.0
28,3786.0
29,3357.0
30,2478.0
32,-1.0


With the rule of 3 last month of sales I've captured all those closed shops.

Items sales are sparse and it'll be challenging to identify seasonal patterns and discontinued products. Perhaps item category can help to aggregate them and better see the trends.

## Items sales

complement items with their categories

In [13]:
items_df = pd.read_csv('../items.csv')

print("Sanity check: known item categories is %d and total categories is %d" %\
      (len(items_df.item_category_id.unique()),len(categories.item_category_id)))

items_df = pd.merge(items_df, categories, how='left', on=['item_category_id'])

items_df.head()

Sanity check: known item categories is 84 and total categories is 84


,item_name,item_id,item_category_id,type_code,subtype_code
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,11,4
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,15,31
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,11,4
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,11,4
4,***КОРОБКА (СТЕКЛО) D,4,40,11,4


In [14]:
# let's check if there are items that are never sold

categories_df = pd.read_csv('../item_categories.csv')

print("Total number of items is %d, among those sold are %d" % \
      (len(items_df.item_id), len(items.item_id)) )

items_never_sold = set(items_df.item_id.unique().tolist()).difference(set(items.item_id))

cat_of_unsold_items = items_df[items_df.item_id.isin(list(items_never_sold))].item_category_id.unique()

type_codes = categories[categories.item_category_id.isin(cat_of_unsold_items)]

print("Never sold are %d items from %d categories of %d types and %d subtypes" %\
      (len(items_never_sold), len(cat_of_unsold_items), len(type_codes.type_code.unique()), len(type_codes.subtype_code.unique())))

print(" (in total there are %d types and %d subtypes)" % (len(categories.type_code.unique()), len(categories.subtype_code.unique())))

categories_df[ categories_df.item_category_id.isin(cat_of_unsold_items)].tail()

Total number of items is 22170, among those sold are 21807
Never sold are 363 items from 39 categories of 12 types and 32 subtypes
 (in total there are 20 types and 65 subtypes)


,item_category_name,item_category_id
72,Подарки - Фигурки,72
75,Программы - Для дома и офиса,75
76,Программы - Для дома и офиса (Цифра),76
77,Программы - Обучающие,77
78,Программы - Обучающие (Цифра),78


Simplest strategy is to assume that items never sold will not also be ever sold

In [15]:
gapless_items_extended = pd.merge(gapless_items, items_df, how='left', on=['item_id'])
gapless_items_extended.tail()

,item_id,date_block_num,target_sum,target_len,target_mean,item_name,item_category_id,type_code,subtype_code
763240,22169,30,NaN,NaN,NaN,Яйцо дракона (Игра престолов),69,14,56
763241,22169,31,NaN,NaN,NaN,Яйцо дракона (Игра престолов),69,14,56
763242,22169,32,NaN,NaN,NaN,Яйцо дракона (Игра престолов),69,14,56
763243,22169,33,NaN,NaN,NaN,Яйцо дракона (Игра престолов),69,14,56
763244,22169,34,NaN,NaN,NaN,Яйцо дракона (Игра престолов),69,14,56


there are ~22K items, most are very sparse; check just a best selling one

In [16]:
print(len(gapless_items.item_id.unique()))
bestseller = gapless_items.iloc[np.argmax(gapless_items.target_mean),0]
gapless_items[gapless_items.item_id == bestseller].tail()

21807


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


,item_id,date_block_num,target_sum,target_len,target_mean
390980,11373,30,201.0,1.0,201.0
390981,11373,31,75.0,1.0,75.0
390982,11373,32,246.0,1.0,246.0
390983,11373,33,2253.0,1.0,2253.0
390984,11373,34,NaN,NaN,NaN


Let now look into time series of sales aggregated by category, type_code, and subtype_code

In [17]:
sales_cats = (gapless_items_extended
              .fillna(0)
              .groupby(['item_category_id','date_block_num'])
              .agg({'target_sum': sum, 'target_len': sum}) # with the sum you are immune to gaps
              .reset_index())

sales_cats['target_mean'] = sales_cats['target_sum']/sales_cats['target_len']

sales_cats.fillna(0, inplace=True)

sales_cats.head()

,item_category_id,date_block_num,target_sum,target_len,target_mean
0,0,0,1.0,1.0,1.0
1,0,1,1.0,1.0,1.0
2,0,2,0.0,0.0,0.0
3,0,3,1.0,1.0,1.0
4,0,4,0.0,0.0,0.0


In [18]:
def construct_lags(sales,cols):
    """
    sales - two dimensional data frame
    """
    sales_lag1 = pd.DataFrame(np.dot(lag1, sales.values), columns=cols)
    sales_lag2 = pd.DataFrame(np.dot(lag2, sales.values), columns=cols)
    sales_lag3 = pd.DataFrame(np.dot(lag3, sales.values), columns=cols)

    no_sales = (sales_lag1 + sales_lag2 + sales_lag3 < 1).stack().reset_index()
    no_sales.columns = ['date_block_num', sales.columns.name, 'no_sales']

    lags = pd.concat([sales_lag1.stack(), sales_lag2.stack(), sales_lag3.stack()], axis=1)
    lags.reset_index(inplace=True)
    lags.columns = ['date_block_num', sales.columns.name, 'lag1', 'lag2', 'lag3']
    return pd.merge(no_sales, lags, how='left', on=['date_block_num', sales.columns.name])

In [19]:
no_sales_shops = construct_lags(sales_shops, sorted(sales_shops.columns.unique().tolist()))

sales_shops = (gapless_shops
               .pivot(columns='shop_id', values='target_mean', index='date_block_num')
               .fillna(0))
lagged_sales_shops = construct_lags(sales_shops, sorted(sales_shops.columns.unique().tolist()))
lagged_sales_shops['no_sales'] = no_sales_shops['no_sales']

lagged_sales_shops.sort_values(by=['shop_id','date_block_num']).head()

,date_block_num,shop_id,no_sales,lag1,lag2,lag3
0,0,0,True,0.000000,0.000000,0.000000
60,1,0,False,2.338784,0.000000,0.000000
120,2,0,False,2.515189,2.338784,0.000000
180,3,0,False,0.000000,2.515189,2.338784
240,4,0,False,0.000000,0.000000,2.515189


In [20]:
sales_cats_sum = sales_cats.pivot(columns='item_category_id', values='target_sum', index='date_block_num')
no_sales_cats = construct_lags(sales_cats_sum, sorted(sales_cats_sum.columns.unique().tolist()))

sales_cats_mean = sales_cats.pivot(columns='item_category_id', values='target_mean', index='date_block_num')

lagged_sales_cats = construct_lags(sales_cats_mean, sorted(sales_cats_mean.columns.unique().tolist()))
lagged_sales_cats['no_sales'] = no_sales_cats['no_sales']

lagged_sales_cats.sort_values(by=['item_category_id','date_block_num']).tail()

,date_block_num,item_category_id,no_sales,lag1,lag2,lag3
2603,30,83,False,2.965517,3.126437,2.936170
2687,31,83,False,3.204301,2.965517,3.126437
2771,32,83,False,2.989796,3.204301,2.965517
2855,33,83,False,2.855670,2.989796,3.204301
2939,34,83,False,3.411765,2.855670,2.989796


In [21]:
sales_types = (gapless_items_extended
              .fillna(0)
              .groupby(['type_code','date_block_num'])
              .agg({'target_sum': sum, 'target_len': sum})
              .reset_index())

sales_types['target_mean'] = sales_types['target_sum']/sales_types['target_len']
sales_types.fillna(0,inplace=True)

sales_types_sum = sales_types.pivot(columns='type_code', values='target_sum', index='date_block_num')
no_sales_types = construct_lags(sales_types_sum, sorted(sales_types_sum.columns.unique().tolist()))

sales_types_mean = sales_types.pivot(columns='type_code', values='target_mean', index='date_block_num')
lagged_sales_types = construct_lags(sales_types_mean, sorted(sales_types_mean.columns.unique().tolist()))
lagged_sales_types['no_sales'] = no_sales_types['no_sales']

lagged_sales_types.tail()

,date_block_num,type_code,no_sales,lag1,lag2,lag3
695,34,15,False,2.474548,2.299197,2.257511
696,34,16,False,13.358974,44.409449,9.690000
697,34,17,True,0.000000,0.000000,0.000000
698,34,18,True,0.000000,0.000000,0.000000
699,34,19,False,3.411765,2.855670,2.989796


In [22]:
sales_subtypes = (gapless_items_extended
                  .fillna(0)
                  .groupby(['subtype_code','date_block_num'])
                  .agg({'target_sum': sum, 'target_len': sum})
                  .reset_index())

sales_subtypes['target_mean'] = sales_subtypes['target_sum']/sales_subtypes['target_len']
sales_subtypes.fillna(0, inplace=True)

sales_subtypes_sum = sales_subtypes.pivot(columns='subtype_code', values='target_sum', index='date_block_num')
no_sales_subtypes = construct_lags(sales_subtypes_sum, sorted(sales_subtypes_sum.columns.unique().tolist()))

sales_subtypes_mean = sales_subtypes.pivot(columns='subtype_code', values='target_mean', index='date_block_num')
lagged_sales_subtypes = construct_lags(sales_subtypes_mean, sorted(sales_subtypes_mean.columns.unique().tolist()))
lagged_sales_subtypes['no_sales'] = no_sales_subtypes['no_sales']

lagged_sales_subtypes.tail()

,date_block_num,subtype_code,no_sales,lag1,lag2,lag3
2270,34,60,True,0.000000,0.000000,0.000000
2271,34,61,False,4.383901,3.533537,3.118750
2272,34,62,True,0.000000,0.000000,0.000000
2273,34,63,True,0.000000,0.000000,0.000000
2274,34,64,False,3.411765,2.855670,2.989796


In [23]:
print("No sales for %d categories, %d types, and %d subtypes" %\
      (sum(lagged_sales_cats[lagged_sales_cats.date_block_num==34].no_sales),
       sum(lagged_sales_types[lagged_sales_types.date_block_num==34].no_sales),
       sum(lagged_sales_subtypes[lagged_sales_subtypes.date_block_num==34].no_sales)))

No sales for 19 categories, 4 types, and 15 subtypes


Should I try to zero those out as well?

In [24]:
# replace NAs for 0 sales in the [date,item] sales matrix

sales_items_sum = gapless_items_extended.pivot(columns='item_id', values='target_sum', index='date_block_num')
sales_items_sum.fillna(0, inplace=True)
no_sales_items = construct_lags(sales_items_sum, sorted(sales_items_sum.columns.unique().tolist()))

sales_items_mean = gapless_items_extended.pivot(columns='item_id', values='target_mean', index='date_block_num')
sales_items_mean.fillna(0, inplace=True)
lagged_sales_items = construct_lags(sales_items_mean, sorted(sales_items_mean.columns.unique().tolist()))
lagged_sales_items['no_sales'] = no_sales_items['no_sales']

lagged_sales_items.tail()

,date_block_num,item_id,no_sales,lag1,lag2,lag3
763240,34,22165,True,0.000000,0.0,0.0
763241,34,22166,False,11.000000,5.0,11.0
763242,34,22167,False,2.846154,3.5,2.9
763243,34,22168,True,0.000000,0.0,0.0
763244,34,22169,True,0.000000,0.0,0.0


Anyways, before lumpnig these things together, let's find seasonal component when possible.
Before doing so run a trivial sanity check:

In [25]:
# items discontinued with no sales in the last 3 months
discontinued_items = no_sales_items[(no_sales_items.date_block_num==34)&(no_sales_items.no_sales==True)].item_id.tolist()

print("%d out of %d items were not sold at least last 3 months" % \
      (sum(no_sales_items[no_sales_items.date_block_num==34].no_sales),len(items)))

# several items, {15106, 13466, 5613, 16689, 9242, 20539},
#  with negative sales (returns) are also considered to be out of stock over last 3 months
monthly[monthly.item_id==20539].groupby(['date_block_num']).agg({'target': sum}).tail()

14627 out of 21807 items were not sold at least last 3 months


,target
date_block_num,
29,18.0
31,-1.0


In [26]:
# sanity check: filtering last months will yield only items in stock and shops open 
print(
  set(monthly[monthly.date_block_num==32].shop_id.unique()).intersection(set(closed_shops))
)
print(
  set(monthly[monthly.date_block_num==31].item_id.unique()).intersection(set(discontinued_items))
)

{27}
{16689, 15106, 20539, 5613}


these are just the id of returns from above. Although, there is still seasonality to explore, let's see how well I am doing already with the mean encodings and their lags.

In [27]:
lagged_sales_shops = lagged_sales_shops.sort_values(by=['shop_id','date_block_num'])

lagged_sales_shops.head()

,date_block_num,shop_id,no_sales,lag1,lag2,lag3
0,0,0,True,0.000000,0.000000,0.000000
60,1,0,False,2.338784,0.000000,0.000000
120,2,0,False,2.515189,2.338784,0.000000
180,3,0,False,0.000000,2.515189,2.338784
240,4,0,False,0.000000,0.000000,2.515189


In [28]:
total_sales_shops = (gapless_shops
                     .fillna(0)
                     .groupby(['shop_id'])
                     .agg({'target_sum': sum, 'target_len': sum})
                     .reset_index())

total_sales_shops['trg_shop'] = total_sales_shops['target_sum']/total_sales_shops['target_len']

#pd.merge(lagged_sales_shops, total_sales_shops, how='left', on=['shop_id'])

#lagged_sales_cats
lagged_sales_shops

,date_block_num,shop_id,no_sales,lag1,lag2,lag3
0,0,0,True,0.000000,0.000000,0.000000
60,1,0,False,2.338784,0.000000,0.000000
120,2,0,False,2.515189,2.338784,0.000000
180,3,0,False,0.000000,2.515189,2.338784
240,4,0,False,0.000000,0.000000,2.515189
300,5,0,True,0.000000,0.000000,0.000000
360,6,0,True,0.000000,0.000000,0.000000
420,7,0,True,0.000000,0.000000,0.000000
480,8,0,True,0.000000,0.000000,0.000000
540,9,0,True,0.000000,0.000000,0.000000


In [29]:
# shops
lagged_sales_shops.set_index(['date_block_num','shop_id'], inplace=True)
## average over the all of the dates remembering to zero-suppress the means
#total_sales_shops = (gapless_shops
#                     .fillna(0)
#                     .groupby(['shop_id'])
#                     .agg({'target_sum': sum, 'target_len': sum})
#                     .reset_index())
#
#lagged_sales_shops['trg_shop'] 
#
#= total_sales_shops['target_sum'] / total_sales_shops['target_len']
#
#lagged_sales_shops['trg_shop'].fillna(0)

lagged_sales_shops.columns = ['no_sales_shop',
                              'trg_shop_lag1',
                              'trg_shop_lag2',
                              'trg_shop_lag3'] #,
#                              'trg_shop']

# categories
lagged_sales_cats.set_index(['date_block_num','item_category_id'], inplace=True)

#total_sales_cats = (gapless_items_extended
#                    .fillna(0)
#                    .groupby(['item_category_id'])
#                    .agg({'target_sum': sum, 'target_len': sum})
#                    .reset_index())
#
#lagged_sales_cats['trg_cat'] = total_sales_cats['target_sum'] / total_sales_cats['target_len']
#lagged_sales_cats['trg_cat'].fillna(0)

lagged_sales_cats.columns = ['no_sales_cat',
                             'trg_cat_lag1',
                             'trg_cat_lag2',
                             'trg_cat_lag3']#,
#                             'trg_cat']

# subtypes
lagged_sales_subtypes.set_index(['date_block_num','subtype_code'], inplace=True)

#total_sales_subtypes = (gapless_items_extended
#                        .fillna(0)
#                        .groupby(['subtype_code'])
#                        .agg({'target_sum': sum, 'target_len': sum})
#                        .reset_index())
#
#lagged_sales_subtypes['trg_subtype'] = total_sales_subtypes['target_sum'] / total_sales_subtypes['target_len']
#lagged_sales_subtypes['trg_subtype'].fillna(0)

lagged_sales_subtypes.columns = ['no_sales_subtype',
                                 'trg_subtype_lag1',
                                 'trg_subtype_lag2',
                                 'trg_subtype_lag3']#,
#                                 'trg_subtype']

# types
lagged_sales_types.set_index(['date_block_num','type_code'], inplace=True)

#total_sales_types = (gapless_items_extended
#                     .fillna(0)
#                     .groupby(['type_code'])
#                     .agg({'target_sum': sum, 'target_len': sum})
#                     .reset_index())
#
#lagged_sales_types['trg_type'] = total_sales_types['target_sum'] / total_sales_types['target_len']
#lagged_sales_types['trg_type'].fillna(0)

lagged_sales_types.columns = ['no_sales_type',
                              'trg_type_lag1',
                              'trg_type_lag2',
                              'trg_type_lag3']#,
#                              'trg_type']

# items
sales_items = gapless_items.fillna(0).set_index(['date_block_num','item_id'])['target_mean']
lagged_sales_items.set_index(['date_block_num','item_id'], inplace=True)

#total_sales_items = (gapless_items_extended
#                     .fillna(0)
#                     .groupby(['item_id'])
#                     .agg({'target_sum': sum, 'target_len': sum})
#                     .reset_index())
#
#lagged_sales_items['trg_item'] = total_sales_items['target_sum'] / total_sales_items['target_len']
#lagged_sales_items['trg_item'].fillna(0)

lagged_sales_items.columns = ['no_sales_item',
                              'trg_item_lag1',
                              'trg_item_lag2',
                              'trg_item_lag3']#,
#                              'trg_item']

In [30]:
lagged_sales_shops.reset_index()
all_data = pd.merge(all_data, lagged_sales_shops, how='left', on=['shop_id','date_block_num'])

lagged_sales_cats.reset_index()
all_data = pd.merge(all_data, lagged_sales_cats,  how='left', on=['item_category_id','date_block_num'])

lagged_sales_types.reset_index()
all_data = pd.merge(all_data, lagged_sales_types, how='left', on=['type_code','date_block_num'])

lagged_sales_subtypes.reset_index()
all_data = pd.merge(all_data, lagged_sales_subtypes, how='left', on=['subtype_code','date_block_num'])

lagged_sales_items.reset_index()
all_data = pd.merge(all_data, lagged_sales_items, how='left', on=['item_id','date_block_num'])

In [31]:
#sales_shops.stack().index
#lagged_sales_shops.reset_index(inplace=True)
#all_data.loc[,['total_monthly_category_sales_lag_1', 'trg_cat_lag1']]

#sales_shops.stack().index
#lagged_sales_shops.reset_index(inplace=True)
#all_data.loc[11128043,].head()

all_data[ (all_data.item_id == 18049) & (all_data.shop_id == 45) & (all_data.date_block_num > 30) ]

,item_id,shop_id,date_block_num,target,item_category_id,type_code,subtype_code,item_name,total_monthly_sales,total_monthly_shop_sales,...,trg_type_lag2,trg_type_lag3,no_sales_subtype,trg_subtype_lag1,trg_subtype_lag2,trg_subtype_lag3,no_sales_item,trg_item_lag1,trg_item_lag2,trg_item_lag3
10389728,18049,45,31,0.0,70,14,57,Резинки для плетения силиконовые Неон желтый N...,0.287989,0.138998,...,2.363620,2.449621,False,1.970803,2.324131,2.770760,False,1.526316,2.750000,2.454545
10608694,18049,45,32,0.0,70,14,57,Резинки для плетения силиконовые Неон желтый N...,0.263758,0.128810,...,2.318013,2.363620,False,1.818294,1.970803,2.324131,False,1.090909,1.526316,2.750000
10847888,18049,45,33,0.0,70,14,57,Резинки для плетения силиконовые Неон желтый N...,0.258565,0.126732,...,2.211383,2.318013,False,1.831843,1.818294,1.970803,False,1.250000,1.090909,1.526316
11128043,18049,45,34,0.0,70,14,57,Резинки для плетения силиконовые Неон желтый N...,0.000000,0.000000,...,2.311570,2.211383,False,2.596026,1.831843,1.818294,False,1.000000,1.250000,1.090909


In [32]:
#qwe = lagged_sales_shops.reset_index()
#qwe[(qwe.shop_id == 45) & (qwe.date_block_num > 30)]

gapless_shops[(gapless_shops.shop_id == 45) & (gapless_shops.date_block_num > 30)]

,shop_id,date_block_num,target_sum,target_len,target_mean
1606,45,31,710.0,505.0,1.405941
1607,45,32,654.0,410.0,1.595122
1608,45,33,702.0,466.0,1.506438
1609,45,34,NaN,NaN,NaN


In [33]:
np.mean(monthly[(monthly.shop_id == 45) & (monthly.date_block_num == 30)].target)
len(all_data[(all_data.shop_id == 45) & (all_data.date_block_num == 30)].fillna(0).target)
#len(agg[(agg.shop_id == 45) & (agg.date_block_num == 30)].item_id.unique())

5323

In [34]:
predictors = ['item_id', 'shop_id', 'date_block_num', 'item_category_id',
       'type_code', 'subtype_code', 'total_monthly_sales',
       'target_lag_12', 'target_lag_1',
       'target_lag_2', 'target_lag_3', 'target_lag_6',
       'no_sales_shop',
       'trg_shop_lag1', 'trg_shop_lag2', 'trg_shop_lag3',
       'no_sales_cat', 'trg_cat_lag1', 'trg_cat_lag2', 'trg_cat_lag3',
       'no_sales_type', 'trg_type_lag1', 'trg_type_lag2',
       'trg_type_lag3', 'no_sales_subtype', 'trg_subtype_lag1',
       'trg_subtype_lag2', 'trg_subtype_lag3', 'no_sales_item',
       'trg_item_lag1', 'trg_item_lag2', 'trg_item_lag3']

predictors += [
    "total_monthly_sales_lag_1",
    "total_monthly_shop_sales_lag_1",
    "total_monthly_item_sales_lag_1",
    "total_monthly_category_sales_lag_1",
    "total_monthly_supercategory_sales_lag_1",
    "total_monthly_subcategory_sales_lag_1",
    "total_monthly_shop_category_sales_lag_1",
    "total_monthly_shop_supercategory_sales_lag_1",    
    "total_monthly_shop_subcategory_sales_lag_1"]

#predictors = ['item_id', 'shop_id', 'date_block_num', 'item_category_id',
#       'type_code', 'subtype_code', 'item_name', 'total_monthly_sales',
#       'target_lag_12', 'target_lag_1',
#       'target_lag_2', 'target_lag_3', 'target_lag_6',
#       'no_sales_shop',
#       'trg_shop_lag1', 'trg_shop_lag2', 'trg_shop_lag3', 'trg_shop',
#       'no_sales_cat', 'trg_cat_lag1', 'trg_cat_lag2', 'trg_cat_lag3',
#       'trg_cat', 'no_sales_type', 'trg_type_lag1', 'trg_type_lag2',
#       'trg_type_lag3', 'trg_type', 'no_sales_subtype', 'trg_subtype_lag1',
#       'trg_subtype_lag2', 'trg_subtype_lag3', 'trg_subtype', 'no_sales_item',
#       'trg_item_lag1', 'trg_item_lag2', 'trg_item_lag3', 'trg_item']

In [35]:
train = all_data[(all_data.date_block_num>12)&(all_data.date_block_num<34)].fillna(0)

X_train = train[train.date_block_num <  33][predictors]
X_valid = train[train.date_block_num == 33][predictors]
y_train = train[train.date_block_num <  33].target
y_valid = train[train.date_block_num == 33].target

In [49]:
%%time
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=64, max_features=6, n_jobs=6) #len(predictors)/2
model.fit(X_train, y_train)
#from joblib import dump, load
#dump(model,"model1.joblib")

CPU times: user 2.79 s, sys: 20.2 s, total: 22.9 s
Wall time: 35.6 s


['model1.joblib']

In [ ]:
from sklearn.metrics import mean_squared_error

ypred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, ypred))
print("RMSE good: %f" % (rmse))
print("Average good: {0}".format(np.mean(ypred)))

In [38]:
%%time
from sklearn.ensemble import RandomForestRegressor
full_model = RandomForestRegressor(n_estimators=64, max_features=6, n_jobs=6)
full_model.fit(train[predictors], train.target)



CPU times: user 1h 7min 58s, sys: 1min 47s, total: 1h 9min 45s
Wall time: 22min 45s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=6,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [39]:
# combine
month_34 = all_data[all_data.date_block_num == 34].fillna(0)
month_34['item_cnt_month'] = full_model.predict(month_34[predictors])

# automatically set 0 for "bad" items
test = pd.merge(test_df, month_34, how="left", on=["shop_id","item_id"]).fillna(0)

# zeroing out
print(test[test.shop_id.isin(closed_shops)].shape)

suppress = test.item_id.isin(items_never_sold).apply(lambda x: 0 if x else 1)
test.item_cnt_month *= suppress

test = test.loc[:,['ID', 'item_cnt_month']]
test.set_index("ID", inplace=True)
test["item_cnt_month"] = test["item_cnt_month"].round().astype("int64")

# clipping
test[test.item_cnt_month>20] = 20

test.to_csv("item_trends.csv")

(0, 55)


In [40]:
# public score: 1.04069

In [50]:
%macro -q __trends_combined 3-35

In [51]:
%store __trends_combined

Stored '__trends_combined' (Macro)


## Seasonal components

First of all, it totally make sense to introduce some more surrogate variables:
* month of the year (1-12)
* cristmas sale (True/False)
* sales 12 month ago
* current trend

In [41]:
# identify shops and items with sales history of > 1 year (needed for seasonal component)

monthly.groupby(['shop_id']).agg({'date_block_num': [min,max]})


date_block_num    
                   min max
shop_id                   
0                    0   1
1                    0   1
2                    0  33
3                    0  33
4                    0  33
5                    1  33
6                    0  33
7                    0  33
8                    0   3
9                    9  33
10                   0  33
11                  25  25
12                   0  33
13                   0  19
14                   0  33
15                   0  33
16                   0  33
17                   5  24
18                   0  33
19                   0  33
20                  21  33
21                   0  33
22                   0  33
23                   0   3
24                   0  33
25                   0  33
26                   0  33
27                   0  32
28                   0  33
29                   0  28
30                   0  25
31                   0  33
32                   0   6
33                  19  27
34                  18  33
35                   0  33
36                  33  33
37                   0  33
38                   0  33
39                  14  33
40                  14  24
41                   0  33
42                   0  33
43                   0  24
44                   0  33
45                   0  33
46                   0  33
47                   0  33
48                  15  33
49                  11  33
50                   0  33
51                   0  32
52                   0  33
53                   0  33
54                   0  27
55                   4  33
56                   0  33
57                   2  33
58                   2  33
59                   0  33

In [42]:
# Create data frame associating data_block_num with date (January, 2013 is date_block_num==0)
timeline = pd.DataFrame({'begin_date': pd.date_range(start='2013-01-01',end='2015-11-01', freq='MS'),
                         'date_block_num': range(35)} )

timeline.tail()

,begin_date,date_block_num
30,2015-07-01,30
31,2015-08-01,31
32,2015-09-01,32
33,2015-10-01,33
34,2015-11-01,34


In [43]:
#Lags for mean encodings too!